Note that the openvoice conda environment should already have been set up according to the OpenVoice documentation.
```
conda init
conda activate openvoice
pip install torch==2.7.1+cu118 torchaudio==2.7.1+cu118 --index-url https://download.pytorch.org/whl/cu118
```

In [1]:
import os
import sys
sys.path.append(os.path.abspath('OpenVoice'))

In [2]:
import torch
from openvoice import se_extractor
from openvoice.api import ToneColorConverter
from melo.api import TTS

c:\Users\ExoHorizon\miniconda3\envs\openvoice\lib\site-packages\librosa\util\files.py:10: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename
c:\Users\ExoHorizon\miniconda3\envs\openvoice\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



c:\Users\ExoHorizon\miniconda3\envs\openvoice\lib\site-packages\huggingface_hub\file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


An error occurred: module 'importlib.metadata' has no attribute 'packages_distributions'


c:\Users\ExoHorizon\miniconda3\envs\openvoice\lib\site-packages\google\api_core\_python_version_support.py:252: FutureWarning: You are using a Python version (3.9.23) past its end of life. Google will update google.api_core with critical bug fixes on a best-effort basis, but not with any other fixes or features. Please upgrade to the latest Python version, or at least Python 3.10, and then update google.api_core.
  warnings.warn(message, FutureWarning)


In [3]:
import nltk
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\ExoHorizon\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

In [4]:
# Configuration
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print("Using Device:", device)

Using Device: cuda:0


In [5]:
# Initialize ToneColorConverter
ckpt_converter = 'checkpoints_v2/converter'
tone_color_converter = ToneColorConverter(f'OpenVoice/{ckpt_converter}/config.json', device=device)
tone_color_converter.load_ckpt(f'OpenVoice/{ckpt_converter}/checkpoint.pth')

c:\Users\ExoHorizon\miniconda3\envs\openvoice\lib\site-packages\torch\nn\utils\weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


Loaded checkpoint 'OpenVoice/checkpoints_v2/converter/checkpoint.pth'
missing/unexpected keys: [] []


In [24]:
# Paths
reference_speaker_path = 'examples/arnold.wav'
output_dir = 'openvoicev2_outputs'
output_path = os.path.join(output_dir, 'random3_cloned_voice.wav')
os.makedirs(output_dir, exist_ok=True)

In [7]:
# Extract target speaker embedding from reference audio
target_se, audio_name = se_extractor.get_se(reference_speaker_path, tone_color_converter, vad=True)

OpenVoice version: v2
[(0.0, 31.921)]
after vad: dur = 31.921


c:\Users\ExoHorizon\miniconda3\envs\openvoice\lib\site-packages\torch\functional.py:730: UserWarning: stft with return_complex=False is deprecated. In a future pytorch release, stft will return complex tensors for all inputs, and return_complex=False will raise an error.
Note: you can still call torch.view_as_real on the complex output to recover the old return format. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\native\SpectralOps.cpp:880.)
  return _VF.stft(  # type: ignore[attr-defined]


In [8]:
# Print target speaker embedding info
print(f"Audio name: {audio_name}")
print(f"Embedding shape: {target_se.shape}")
print(f"Min value: {target_se.min().item():.6f}")
print(f"Max value: {target_se.max().item():.6f}")
print(f"Mean value: {target_se.mean().item():.6f}")
print(f"Std deviation: {target_se.std().item():.6f}")
print(f"L2 norm: {target_se.norm().item():.6f}")

Audio name: arnold_v2__^Oiy4W4bTghod7pF
Embedding shape: torch.Size([1, 256, 1])
Min value: -4.665708
Max value: 1.389588
Mean value: -0.070411
Std deviation: 0.699831
L2 norm: 11.232040


In [25]:
# Run this to try a random target speaker embedding
target_se = torch.randn_like(target_se)

In [10]:
# English text to synthesize
text = "Did you ever hear a folk tale about a giant turtle?"
src_path = os.path.join(output_dir, 'tmp.wav')
speed = 1.0

In [11]:
# Initialize English TTS model
language = 'EN'
model = TTS(language=language, device=device)
speaker_ids = model.hps.data.spk2id

c:\Users\ExoHorizon\miniconda3\envs\openvoice\lib\site-packages\torch\nn\utils\weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


In [12]:
# Use the first English base speaker
speaker_key = list(speaker_ids.keys())[0]
speaker_id = speaker_ids[speaker_key]
speaker_key = speaker_key.lower().replace('_', '-')

In [ ]:
# Load source speaker embedding
source_se = torch.load(f'OpenVoice/checkpoints_v2/base_speakers/ses/{speaker_key}.pth', map_location=device)

In [14]:
# Optional workaround for MPS on CPU
if torch.backends.mps.is_available() and device == 'cpu':
    torch.backends.mps.is_available = lambda: False

In [15]:
# Generate reference audio using MeloTTS
model.tts_to_file(text, speaker_id, src_path, speed=speed)

 > Text split to sentences.
Did you ever hear a folk tale about a giant turtle?
 > ===========================


  0%|          | 0/1 [00:00<?, ?it/s]c:\Users\ExoHorizon\miniconda3\envs\openvoice\lib\site-packages\huggingface_hub\file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
c:\Users\ExoHorizon\min

In [26]:
# Apply tone color conversion to clone target voice
encode_message = "@MyShell"
tone_color_converter.convert(
    audio_src_path=src_path,
    src_se=source_se,
    tgt_se=target_se,
    output_path=output_path,
    message=encode_message
)